In [1]:
# import libs
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
import os
from tqdm import tqdm
from plotly.subplots import make_subplots
from utils import (
    read_from_ids,
    query,
    relative,
    normalize
)
from datetime import datetime, timedelta
import holidays
import numpy as np
import folium
import umap
import plotly.express as px

In [2]:
# Specify data path
categories = ["workday", "weekend", "holiday"]
data_path = "./preprocessed_data/city/"
id_path = "./preprocessed_data/city/city_id.csv"
geo_sensor_path = "./spatial/loop_update_city/"
geo_district_path =  "./spatial/districts/"

In [3]:
# Plot the city map with sensor locations
city_gdf = gpd.read_file(geo_sensor_path).to_crs("EPSG:4326")
city_geo_list = np.array([[point.xy[1][0], point.xy[0][0]] for point in city_gdf['geometry']])

district_gdf = gpd.read_file(geo_district_path).to_crs("EPSG:4326")
district_gdf['bezeichnun'] = district_gdf['bezeichnun'].apply(lambda x: x.replace(' ', '_'))
district_gdf.drop(columns=['objid', 'name', 'entstehung'], inplace=True)
district_gdf.rename(columns={'bezeichnun': 'Kreis'}, inplace=True)

start_location = np.mean(city_geo_list, axis=0).tolist()
map = folium.Map(location=start_location, tiles="OpenStreetMap", zoom_start=13)

# Add sensor positions
city_sensors = folium.FeatureGroup(name="city_sensors", show=True)

for sensor_position in city_geo_list:
    folium.CircleMarker(location=sensor_position, radius=4, color="orange").add_to(city_sensors)

for _, r in district_gdf.iterrows():
    geo = gpd.GeoSeries(r['geometry'])
    geo_j = geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'blue'})
    folium.Popup(r['Kreis'].replace("_", " ")).add_to(geo_j)
    geo_j.add_to(map)


city_sensors.add_to(map)
folium.LayerControl().add_to(map)
map.save("figs/map.html")

In [4]:
# Plot the meta data
metadata_list = []
for i in range(len(categories)):
    metadata_list.append(pd.read_csv(os.path.join(data_path, categories[i], "metadata.csv"), index_col=0).rename(columns={"0": i}))

metadata_df = pd.concat(metadata_list, axis=1)

layout = dict(
    title = "Metadata",
    autosize = True,
    width = 1000,
    height = 400
)

fig = go.Figure(
    data=go.Table(
        header=dict(values = list(metadata_df.index)),
        cells=dict(values=list(metadata_df.to_numpy()))
    ),
    layout=layout
)

fig.write_html("figs/metadata.html")


In [4]:
# Data path
id = list(pd.read_csv(id_path)["detid"])

# Load the data 
workday_df_dict = read_from_ids(id, "workday", data_path)
weekend_df_dict = read_from_ids(id, "weekend", data_path)
holiday_df_dict = read_from_ids(id, "holiday", data_path)
workday_df_list = list(workday_df_dict.values())
weekend_df_list = list(weekend_df_dict.values())
holiday_df_list = list(holiday_df_dict.values())

years = [2018, 2019, 2020, 2021]
months = list(range(1,13))
month_names = ["January", "Febrary", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

Reading holiday data: 100%|██████████| 1646/1646 [00:06<00:00, 272.09it/s]


In [ ]:
# Daily occupancy averaged over one month over all sensors for workdays
steps = []
fig = make_subplots(rows=2 ,cols=2)
n_plot = 0
for year in tqdm(years, desc="Looping years", position=1):
    for month in tqdm(months, desc="Looping months", position=0):
        end_year = year + 1 if month == 12 else year
        end_month = 1 if month == 12 else month + 1
        df = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
        fig.add_trace(
            go.Bar(
                name = f"{year}-{month_names[month-1]}",
                x = df["hour"],
                y = df["occ"],
                
            ),
            row = n_plot // 2 + 1,
            col = n_plot % 2 + 1
        )
    n_plot += 1

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * len(years) * len(months)},
            {"title": month_names[month] + " Average Daily Occupancy for Workdays"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month] = True  
    step["args"][0]["visible"][month + 12] = True  
    step["args"][0]["visible"][month + 24] = True  
    step["args"][0]["visible"][month + 36] = True  
    steps.append(step)
    

sliders = [dict(
    active=1,
    steps=steps
)]


fig.update_layout(
    sliders=sliders,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
)
fig.update_yaxes(range = [0, 4])
fig.write_html("figs/workday_daily.html")


In [ ]:
# Daily occupancy averaged over one month over all sensors for workdays
steps = []
fig = make_subplots(rows=2 ,cols=2)
n_plot = 0
for year in tqdm(years, desc="Looping years", position=1):
    for month in tqdm(months, desc="Looping months", position=0):
        end_year = year + 1 if month == 12 else year
        end_month = 1 if month == 12 else month + 1
        df = query(weekend_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
        fig.add_trace(
            go.Bar(
                name = f"{year}-{month_names[month-1]}",
                x = df["hour"],
                y = df["occ"],
                
            ),
            row = n_plot // 2 + 1,
            col = n_plot % 2 + 1
        )
    n_plot += 1

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * len(years) * len(months)},
            {"title": month_names[month] + " Average Daily Occupancy for Workdays"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month] = True  
    step["args"][0]["visible"][month + 12] = True  
    step["args"][0]["visible"][month + 24] = True  
    step["args"][0]["visible"][month + 36] = True  
    steps.append(step)
    

sliders = [dict(
    active=1,
    steps=steps
)]


fig.update_layout(
    sliders=sliders,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
)
fig.update_yaxes(range = [0, 4])
fig.write_html("figs/weekend_daily.html")

In [ ]:
# Daily occupancy averaged over all sensors for holidays 
steps = []
fig = make_subplots(rows=2 ,cols=2)
n_plot = 0
for year in tqdm(years, desc="Looping years", position=1):
    ch_zh_holidays = holidays.CH(subdiv="ZH", years=[year])
    for holiday_date, holiday_name in tqdm(ch_zh_holidays.items(), desc="Looping holidays", position=0):
        holiday_date = datetime(holiday_date.year, holiday_date.month, holiday_date.day)
        df = query(holiday_df_list, holiday_date, holiday_date+timedelta(days=1), "hour")
        fig.add_trace(
            go.Bar(
                name = f"{year}-{holiday_name}",
                x = df["hour"],
                y = df["occ"],
                
            ),
            row = n_plot // 2 + 1,
            col = n_plot % 2 + 1
        )
    n_plot += 1

ch_zh_holidays = holidays.CH(subdiv="ZH", years=[2018])
n_holidays = len(ch_zh_holidays)
i=0
for holiday_date, holiday_name in ch_zh_holidays.items(): 
    holiday_name = ch_zh_holidays[holiday_date]
    step = dict(
        method="update",
        args=[{"visible": [False] * len(ch_zh_holidays.keys()) * len(years)},
            {"title": holiday_name + " Average Daily Occupancy"}],  # layout attribute
        label= f"{month_names[holiday_date.month - 1]} - {holiday_date.day} {holiday_name}"
    )
    step["args"][0]["visible"][i] = True  
    step["args"][0]["visible"][i + n_holidays] = True  
    step["args"][0]["visible"][i + n_holidays * 2] = True  
    step["args"][0]["visible"][i + n_holidays * 3] = True  
    steps.append(step)
    i += 1
    

sliders = [dict(
    active=1,
    steps=steps,
    currentvalue = {}
)]


fig.update_layout(
    sliders=sliders,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
)
fig.update_yaxes(range = [0, 4])
fig.write_html("figs/holiday_daily.html")

In [ ]:
# Density map for monthly average occupancy for every sensor
steps = []
data = []
layout = dict(
    title = "Average daily occupancy at Zurich city",
    width = 1600,
    height = 1000
)

city_gdf_indexed = city_gdf.set_index("detid")
# print(city_gdf_indexed)
n_plot = 0
for year in years:
    lat = []
    lon = []
    sensor_data_list = []
    for detid in tqdm(city_gdf["detid"], desc="Collecting data for every sensor"):
        if detid in list(weekend_df_dict.keys()):
            sensor_data_list.append(query([weekend_df_dict[detid]], datetime(year,1,1), datetime(year+1,1,1), "month")["occ"])
            lat.append(city_gdf_indexed.loc[detid]["geometry"].xy[1][0])
            lon.append(city_gdf_indexed.loc[detid]["geometry"].xy[0][0])

    sensor_data_df = pd.concat(sensor_data_list, axis = 1)
    # print(sensor_data_df)
    z = sensor_data_df.to_numpy()

    for month in months:
        month = month - 1
        data.append(
            go.Densitymapbox(
                lat = lat,
                lon = lon,
                z = z[month, :],
                radius = 10,
                opacity = 1,
                # zauto = True,
                zmin = 0,
                zmax = 10,
                subplot= "mapbox" if n_plot == 0 else f"mapbox{n_plot+1}",
                colorscale= "Jet",
                name = f"{year}-{month_names[month]}"
            )
        )
    n_plot += 1

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * len(months) * len(years)},
            {"title": month_names[month] + " Average Daily Occupancy"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month] = True  
    step["args"][0]["visible"][month + 12] = True  
    step["args"][0]["visible"][month + 24] = True  
    step["args"][0]["visible"][month + 36] = True  
    steps.append(step)

sliders = [dict(
    active=1,
    steps=steps
)]

layout["mapbox"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.05, 0.45], y = [0.55, 0.95] ),
    style = "open-street-map"
)
layout["mapbox2"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.55, 0.95], y = [0.55, 0.95] ),
    style = "open-street-map"
)
layout["mapbox3"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.05, 0.45], y = [0.05, 0.45] ),
    style = "open-street-map"
)
layout["mapbox4"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.55, 0.95], y = [0.05, 0.45] ),
    style = "open-street-map"
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    sliders = sliders
)
fig.write_html("figs/workday_spatial.html")

In [ ]:
# Daily occupancy averaged over one month over all sensors for workdays
steps = []
fig = go.Figure() 
n_plot = 0
for year in tqdm(years, desc="Looping years", position=1):
    df_list = []
    for month in tqdm(months, desc="Looping months", position=0):
        end_year = year + 1 if month == 12 else year
        end_month = 1 if month == 12 else month + 1
        df_list.append(query(workday_df_list, datetime(year,month,1), datetime(end_year,end_month,1), "month"))
        df = pd.concat(df_list, ignore_index=True)
    fig.add_trace(
        go.Bar(
            name = f"{year}",
            x = month_names,
            y = df["occ"],   
        )
    )

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    title = "Average Monthly Occupancy "
)
fig.write_html("figs/workday_monthly.html")

In [ ]:
# Daily flow difference between 2020 and 2018
# Daliy flow difference between 2021 and 2018
steps = []
fig = make_subplots(rows=1, cols=3)
for month in tqdm(months, desc="Looping months", position=0):
    end_month = 1 if month == 12 else month + 1
    year = 2018
    end_year = year + 1 if month == 12 else year
    df_1 = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    year = 2019 
    end_year = year + 1 if month == 12 else year
    df_2 = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    fig.add_trace(
        go.Bar(
            name = f"{year}-{month_names[month-1]}",
            x = df_2["hour"],
            y = df_2["occ"] - df_1["occ"],
        ),
        row = 1,
        col = 1
    )
    year = 2020 
    end_year = year + 1 if month == 12 else year
    df_3 = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    fig.add_trace(
        go.Bar(
            name = f"{year}-{month_names[month-1]}",
            x = df_3["hour"],
            y = df_3["occ"] - df_1["occ"],
        ),
        row = 1,
        col = 2
    )
    year = 2021
    end_year = year + 1 if month == 12 else year
    df_4 = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    fig.add_trace(
        go.Bar(
            name = f"{year}-{month_names[month-1]}",
            x = df_4["hour"],
            y = df_4["occ"] - df_1["occ"],
        ),
        row = 1,
        col = 3
    )

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * 3 * len(months)},
            {"title": month_names[month] + " Average Daily Occupancy Difference Compared with 2018"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month*3] = True  
    step["args"][0]["visible"][month*3+1] = True  
    step["args"][0]["visible"][month*3+2] = True  
    steps.append(step)
    

sliders = [dict(
    active=1,
    steps=steps
)]


fig.update_layout(
    sliders=sliders,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
)


fig.update_yaxes(range = [-1, 1])
fig.write_html("figs/workday_daily_comparison.html")


In [ ]:
# Density map for monthly average occupancy for every sensor
steps = []
data = []
layout = dict(
    title = "Average daily occupancy at Zurich city",
    width = 1600,
    height = 1000
)

city_gdf_indexed = city_gdf.set_index("detid")

lat = []
lon = []
sensor_data_list = []
for detid in tqdm(city_gdf["detid"], desc="Collecting data for every sensor"):
    if detid in list(weekend_df_dict.keys()): 
        sensor_data_list.append(query([weekend_df_dict[detid]], datetime(2018,1,1), datetime(2019,1,1), "month")["occ"])
        lat.append(city_gdf_indexed.loc[detid]["geometry"].xy[1][0])
        lon.append(city_gdf_indexed.loc[detid]["geometry"].xy[0][0])
sensor_data_df_0 = pd.concat(sensor_data_list, axis = 1)
z_0 = sensor_data_df_0.to_numpy()

n_plot = 0
for year in [2019,2020,2021]:
    lat = []
    lon = []
    sensor_data_list = []
    for detid in tqdm(city_gdf["detid"], desc="Collecting data for every sensor"):
        if detid in list(weekend_df_dict.keys()):
            sensor_data_list.append(query([weekend_df_dict[detid]], datetime(year,1,1), datetime(year+1,1,1), "month")["occ"])
            lat.append(city_gdf_indexed.loc[detid]["geometry"].xy[1][0])
            lon.append(city_gdf_indexed.loc[detid]["geometry"].xy[0][0])
    sensor_data_df = pd.concat(sensor_data_list, axis = 1)
    z = sensor_data_df.to_numpy()

    for month in months:
        month = month - 1
        data.append(
            go.Densitymapbox(
                lat = lat,
                lon = lon,
                z = z[month, :] - z_0[month, :],
                radius = 4,
                opacity = 1,
                # zauto = True,
                zmin = -0.1,
                zmax = 0.1,
                subplot= "mapbox" if n_plot == 0 else f"mapbox{n_plot+1}",
                colorscale= "Portland",
                name = f"{year}-{month_names[month]}"
            )
        )
    n_plot += 1

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * len(months) * len(years)},
            {"title": month_names[month] + " Average Daily Occupancy Compared with 2018"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month] = True  
    step["args"][0]["visible"][month + 12] = True  
    step["args"][0]["visible"][month + 24] = True  
    step["args"][0]["visible"][month + 36] = True  
    steps.append(step)

sliders = [dict(
    active=1,
    steps=steps
)]

layout["mapbox"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.05, 0.45], y = [0.55, 0.95] ),
    style = "open-street-map"
)
layout["mapbox2"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.55, 0.95], y = [0.55, 0.95] ),
    style = "open-street-map"
)
layout["mapbox3"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.05, 0.45], y = [0.05, 0.45] ),
    style = "open-street-map"
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    sliders = sliders
)
fig.write_html("figs/workday_spatial_comparison.html")

In [ ]:
# Daily relative occupancy averaged over one month over all sensors for workdays
steps = []
fig = make_subplots(rows=2 ,cols=2)
n_plot = 0
for year in tqdm(years, desc="Looping years", position=1):
    for month in tqdm(months, desc="Looping months", position=0):
        end_year = year + 1 if month == 12 else year
        end_month = 1 if month == 12 else month + 1
        df = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
        fig.add_trace(
            go.Bar(
                name = f"{year}-{month_names[month-1]}",
                x = df["hour"],
                y = to_relative(df["occ"]),
                
            ),
            row = n_plot // 2 + 1,
            col = n_plot % 2 + 1
        )
    n_plot += 1

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * len(years) * len(months)},
            {"title": month_names[month] + " Average Relative Daily Occupancy for Workdays"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month] = True  
    step["args"][0]["visible"][month + 12] = True  
    step["args"][0]["visible"][month + 24] = True  
    step["args"][0]["visible"][month + 36] = True  
    steps.append(step)
    

sliders = [dict(
    active=1,
    steps=steps
)]


fig.update_layout(
    sliders=sliders,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width = 1600,
    height = 1000
)

fig.update_yaxes(range = [0, 0.1])
fig.write_html("figs/workday_daily_relative.html")


In [ ]:

# Daily occupancy averaged over one month over all sensors for weekends
steps = []
fig = make_subplots(rows=2 ,cols=2)
n_plot = 0
for year in tqdm(years, desc="Looping years", position=1):
    for month in tqdm(months, desc="Looping months", position=0):
        end_year = year + 1 if month == 12 else year
        end_month = 1 if month == 12 else month + 1
        df = query(weekend_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
        fig.add_trace(
            go.Bar(
                name = f"{year}-{month_names[month-1]}",
                x = df["hour"],
                y = relative(df["occ"]),
                
            ),
            row = n_plot // 2 + 1,
            col = n_plot % 2 + 1
        )
    n_plot += 1

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * len(years) * len(months)},
            {"title": month_names[month] + " Average Relative Daily Occupancy for Weekends"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month] = True  
    step["args"][0]["visible"][month + 12] = True  
    step["args"][0]["visible"][month + 24] = True  
    step["args"][0]["visible"][month + 36] = True  
    steps.append(step)
    

sliders = [dict(
    active=1,
    steps=steps
)]


fig.update_layout(
    sliders=sliders,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width = 1600,
    height = 1000
)
fig.update_yaxes(range = [0, 0.1])
fig.write_html("figs/weekend_daily_relative.html")

In [ ]:

# Daily flow difference between 2020 and 2018
# Daliy flow difference between 2021 and 2018
steps = []
fig = make_subplots(rows=1, cols=3)
for month in tqdm(months, desc="Looping months", position=0):
    end_month = 1 if month == 12 else month + 1
    year = 2018
    end_year = year + 1 if month == 12 else year
    df_1 = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    year = 2019 
    end_year = year + 1 if month == 12 else year
    df_2 = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    fig.add_trace(
        go.Bar(
            name = f"{year}-{month_names[month-1]}",
            x = df_2["hour"],
            y = relative(df_2["occ"].to_numpy()) - relative(df_1["occ"].to_numpy()),
        ),
        row = 1,
        col = 1
    )
    year = 2020 
    end_year = year + 1 if month == 12 else year
    df_3 = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    fig.add_trace(
        go.Bar(
            name = f"{year}-{month_names[month-1]}",
            x = df_3["hour"],
            y = relative(df_3["occ"].to_numpy()) - relative(df_1["occ"].to_numpy()),
        ),
        row = 1,
        col = 2
    )
    year = 2021
    end_year = year + 1 if month == 12 else year
    df_4 = query(workday_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    fig.add_trace(
        go.Bar(
            name = f"{year}-{month_names[month-1]}",
            x = df_4["hour"],
            y = relative(df_4["occ"].to_numpy()) - relative(df_1["occ"].to_numpy()),
        ),
        row = 1,
        col = 3
    )

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * 3 * len(months)},
            {"title": month_names[month] + " Workday Average Daily Relative Occupancy Difference Compared with 2018"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month*3] = True  
    step["args"][0]["visible"][month*3+1] = True  
    step["args"][0]["visible"][month*3+2] = True  
    steps.append(step)
    

sliders = [dict(
    active=1,
    steps=steps
)]


fig.update_layout(
    sliders=sliders,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width = 1600,
    height = 1000
)



fig.update_yaxes(range = [-0.01, 0.01])
fig.write_html("figs/workday_daily_relative_comparison.html")

In [ ]:


# Daily flow difference between 2020 and 2018
# Daliy flow difference between 2021 and 2018
steps = []
fig = make_subplots(rows=1, cols=3)
for month in tqdm(months, desc="Looping months", position=0):
    end_month = 1 if month == 12 else month + 1
    year = 2018
    end_year = year + 1 if month == 12 else year
    df_1 = query(weekend_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    year = 2019 
    end_year = year + 1 if month == 12 else year
    df_2 = query(weekend_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    fig.add_trace(
        go.Bar(
            name = f"{year}-{month_names[month-1]}",
            x = df_2["hour"],
            y = to_relative(df_2["occ"]) - to_relative(df_1["occ"]),
        ),
        row = 1,
        col = 1
    )
    year = 2020 
    end_year = year + 1 if month == 12 else year
    df_3 = query(weekend_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    fig.add_trace(
        go.Bar(
            name = f"{year}-{month_names[month-1]}",
            x = df_3["hour"],
            y = relative(df_3["occ"].to_numpy()) - relative(df_1["occ"].to_numpy()),
        ),
        row = 1,
        col = 2
    )
    year = 2021
    end_year = year + 1 if month == 12 else year
    df_4 = query(weekend_df_list, datetime(year,month,1), datetime(end_year, end_month, 1), "hour")
    fig.add_trace(
        go.Bar(
            name = f"{year}-{month_names[month-1]}",
            x = df_4["hour"],
            y = relative(df_4["occ"].to_numpy()) - relative(df_1["occ"].to_numpy()),
        ),
        row = 1,
        col = 3
    )

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * 3 * len(months)},
            {"title": month_names[month] + " Workday Average Daily Relative Occupancy Difference Compared with 2018"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month*3] = True  
    step["args"][0]["visible"][month*3+1] = True  
    step["args"][0]["visible"][month*3+2] = True  
    steps.append(step)
    

sliders = [dict(
    active=1,
    steps=steps
)]


fig.update_layout(
    sliders=sliders,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width = 1600,
    height = 1000
)



fig.update_yaxes(range = [-0.01, 0.01])
fig.write_html("figs/weekend_daily_relative_comparison.html")

In [ ]:
# Density map for monthly average occupancy for every sensor
steps = []
data = []
layout = dict(
    title = "Average daily occupancy at Zurich city",
    width = 1600,
    height = 1000
)

city_gdf_indexed = city_gdf.set_index("detid")
# print(city_gdf_indexed)
n_plot = 0
for year in years:
    lat = []
    lon = []
    sensor_data_list = []
    for detid in tqdm(city_gdf["detid"], desc="Collecting data for every sensor"):
        if detid in list(weekend_df_dict.keys()):
            sensor_data_list.append(query([weekend_df_dict[detid]], datetime(year,1,1), datetime(year+1,1,1), "month")["occ"])
            lat.append(city_gdf_indexed.loc[detid]["geometry"].xy[1][0])
            lon.append(city_gdf_indexed.loc[detid]["geometry"].xy[0][0])

    sensor_data_df = pd.concat(sensor_data_list, axis = 1)
    # print(sensor_data_df)
    z = sensor_data_df.to_numpy()

    for month in months:
        month = month - 1
        data.append(
            go.Densitymapbox(
                lat = lat,
                lon = lon,
                z = to_relative_np(z[month, :]),
                radius = 10,
                opacity = 1,
                zmin = 0,
                zmax = 0.01,
                subplot= "mapbox" if n_plot == 0 else f"mapbox{n_plot+1}",
                colorscale= "Jet",
                name = f"{year}-{month_names[month]}"
            )
        )
    n_plot += 1

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * len(months) * len(years)},
            {"title": month_names[month] + " Relative Average Daily Occupancy"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month] = True  
    step["args"][0]["visible"][month + 12] = True  
    step["args"][0]["visible"][month + 24] = True  
    step["args"][0]["visible"][month + 36] = True  
    steps.append(step)

sliders = [dict(
    active=1,
    steps=steps
)]

layout["mapbox"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.05, 0.45], y = [0.55, 0.95] ),
    style = "open-street-map"
)
layout["mapbox2"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.55, 0.95], y = [0.55, 0.95] ),
    style = "open-street-map"
)
layout["mapbox3"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.05, 0.45], y = [0.05, 0.45] ),
    style = "open-street-map"
)
layout["mapbox4"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.55, 0.95], y = [0.05, 0.45] ),
    style = "open-street-map"
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    sliders = sliders
)
fig.write_html("figs/workday_spatial_relative.html")

In [ ]:
# Density map for monthly average occupancy difference for every sensor
steps = []
data = []
layout = dict(
    title = "Average daily occupancy at Zurich city",
    width = 1600,
    height = 1000
)

city_gdf_indexed = city_gdf.set_index("detid")

lat = []
lon = []
sensor_data_list = []
for detid in tqdm(city_gdf["detid"], desc="Collecting data for every sensor"):
    if detid in list(weekend_df_dict.keys()): 
        sensor_data_list.append(query([weekend_df_dict[detid]], datetime(2018,1,1), datetime(2019,1,1), "month")["occ"])
        lat.append(city_gdf_indexed.loc[detid]["geometry"].xy[1][0])
        lon.append(city_gdf_indexed.loc[detid]["geometry"].xy[0][0])
sensor_data_df_0 = pd.concat(sensor_data_list, axis = 1)
z_0 = sensor_data_df_0.to_numpy()

n_plot = 0
for year in [2019,2020,2021]:
    lat = []
    lon = []
    sensor_data_list = []
    for detid in tqdm(city_gdf["detid"], desc="Collecting data for every sensor"):
        if detid in list(weekend_df_dict.keys()):
            sensor_data_list.append(query([weekend_df_dict[detid]], datetime(year,1,1), datetime(year+1,1,1), "month")["occ"])
            lat.append(city_gdf_indexed.loc[detid]["geometry"].xy[1][0])
            lon.append(city_gdf_indexed.loc[detid]["geometry"].xy[0][0])
    sensor_data_df = pd.concat(sensor_data_list, axis = 1)
    z = sensor_data_df.to_numpy()

    for month in months:
        month = month - 1
        data.append(
            go.Densitymapbox(
                lat = lat,
                lon = lon,
                z = relative(z[month, :]) - relative(z_0[month, :]),
                radius = 4,
                opacity = 1,
                # zauto = True,
                zmin = -5*1e-5,
                zmax = 5*1e-5,
                subplot= "mapbox" if n_plot == 0 else f"mapbox{n_plot+1}",
                colorscale= "Portland",
                name = f"{year}-{month_names[month]}"
            )
        )
    n_plot += 1

for month in months:
    month = month - 1
    step = dict(
        method="update",
        args=[{"visible": [False] * len(months) * len(years)},
            {"title": month_names[month] + " Average Daily Occupancy Compared with 2018"}],  # layout attribute
        label=month_names[month]
    )
    step["args"][0]["visible"][month] = True  
    step["args"][0]["visible"][month + 12] = True  
    step["args"][0]["visible"][month + 24] = True  
    step["args"][0]["visible"][month + 36] = True  
    steps.append(step)

sliders = [dict(
    active=1,
    steps=steps
)]

layout["mapbox"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.05, 0.45], y = [0.55, 0.95] ),
    style = "open-street-map"
)
layout["mapbox2"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.55, 0.95], y = [0.55, 0.95] ),
    style = "open-street-map"
)
layout["mapbox3"] = dict(
    center = dict(lat=start_location[0], lon=start_location[1]),
    zoom = 11,
    domain = dict( x = [0.05, 0.45], y = [0.05, 0.45] ),
    style = "open-street-map"
)
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    sliders = sliders
)
fig.write_html("figs/workday_spatial_relative_comparison.html")

In [5]:

reducer = umap.UMAP(n_neighbors=15, # default 15, The size of local neighborhood (in terms of number of neighboring sample points) used for manifold approximation.
          n_components=3, # default 2, The dimension of the space to embed into.
          metric='euclidean', # default 'euclidean', The metric to use to compute distances in high dimensional space.
          n_epochs=1000, # default None, The number of training epochs to be used in optimizing the low dimensional embedding. Larger values result in more accurate embeddings. 
          learning_rate=1.0, # default 1.0, The initial learning rate for the embedding optimization.
          init='spectral', # default 'spectral', How to initialize the low dimensional embedding. Options are: {'spectral', 'random', A numpy array of initial embedding positions}.
          min_dist=0.05, # default 0.1, The effective minimum distance between embedded points.
          spread=1.0, # default 1.0, The effective scale of embedded points. In combination with ``min_dist`` this determines how clustered/clumped the embedded points are.
          low_memory=False, # default False, For some datasets the nearest neighbor computation can consume a lot of memory. If you find that UMAP is failing due to memory constraints consider setting this option to True.
          set_op_mix_ratio=1.0, # default 1.0, The value of this parameter should be between 0.0 and 1.0; a value of 1.0 will use a pure fuzzy union, while 0.0 will use a pure fuzzy intersection.
          local_connectivity=1, # default 1, The local connectivity required -- i.e. the number of nearest neighbors that should be assumed to be connected at a local level.
          repulsion_strength=1.0, # default 1.0, Weighting applied to negative samples in low dimensional embedding optimization.
          negative_sample_rate=5, # default 5, Increasing this value will result in greater repulsive force being applied, greater optimization cost, but slightly more accuracy.
          transform_queue_size=4.0, # default 4.0, Larger values will result in slower performance but more accurate nearest neighbor evaluation.
          a=None, # default None, More specific parameters controlling the embedding. If None these values are set automatically as determined by ``min_dist`` and ``spread``.
          b=None, # default None, More specific parameters controlling the embedding. If None these values are set automatically as determined by ``min_dist`` and ``spread``.
          random_state=42, # default: None, If int, random_state is the seed used by the random number generator;
          metric_kwds=None, # default None) Arguments to pass on to the metric, such as the ``p`` value for Minkowski distance.
          angular_rp_forest=False, # default False, Whether to use an angular random projection forest to initialise the approximate nearest neighbor search.
          target_n_neighbors=-1, # default -1, The number of nearest neighbors to use to construct the target simplcial set. If set to -1 use the ``n_neighbors`` value.
          #target_metric='categorical', # default 'categorical', The metric used to measure distance for a target array is using supervised dimension reduction. By default this is 'categorical' which will measure distance in terms of whether categories match or are different. 
          #target_metric_kwds=None, # dict, default None, Keyword argument to pass to the target metric when performing supervised dimension reduction. If None then no arguments are passed on.
          #target_weight=0.5, # default 0.5, weighting factor between data topology and target topology.
          transform_seed=42, # default 42, Random seed used for the stochastic aspects of the transform operation.
          verbose=False, # default False, Controls verbosity of logging.
          unique=False, # default False, Controls if the rows of your data should be uniqued before being embedded. 
        )


In [6]:
features = pd.read_csv("./preprocessed_data/city/feature.csv")
features["date"] = pd.to_datetime(features["date"], infer_datetime_format=True)
features["year"] = features["date"].apply(lambda x: x.year)


In [7]:
X = features.iloc[:,1:49]
X_trans = reducer.fit_transform(X)

fig = px.scatter_3d(
    x = X_trans[:,0],
    y = X_trans[:,1],
    z = X_trans[:,2],
    color = features["daytype"],
    hover_name = features["date"].astype(str),
)
fig.update_layout(title_text='UMAP',
                showlegend=True,
                legend=dict(orientation="h", yanchor="top", y=0, xanchor="center", x=0.5),
                scene_camera=dict(up=dict(x=0, y=0, z=1), 
                                    center=dict(x=0, y=0, z=-0.1),
                                    eye=dict(x=1.5, y=-1.4, z=0.5)),
                                    margin=dict(l=0, r=0, b=0, t=0),
                scene = dict(xaxis=dict(backgroundcolor='white',
                                        color='black',
                                        gridcolor='#f0f0f0',
                                        title_font=dict(size=10),
                                        tickfont=dict(size=10),
                                        ),
                            yaxis=dict(backgroundcolor='white',
                                        color='black',
                                        gridcolor='#f0f0f0',
                                        title_font=dict(size=10),
                                        tickfont=dict(size=10),
                                        ),
                            zaxis=dict(backgroundcolor='lightgrey',
                                        color='black', 
                                        gridcolor='#f0f0f0',
                                        title_font=dict(size=10),
                                        tickfont=dict(size=10),
                                        )))
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_traces(marker=dict(size=3, line=dict(color='black', width=0.1)))

fig.write_html("figs/umap_daytype.html")


In [9]:
workday_features = features[features["daytype"]=="workday"]
Xwork = workday_features.iloc[:, 1:49]
Xwork_trans = reducer.fit_transform(Xwork)

fig = px.scatter_3d(
    x = Xwork_trans[:,0],
    y = Xwork_trans[:,1],
    z = Xwork_trans[:,2],
    color = workday_features["year"].astype(str),
    hover_name = workday_features["date"].astype(str),
)
fig.update_layout(title_text='UMAP',
                showlegend=True,
                legend=dict(orientation="h", yanchor="top", y=0, xanchor="center", x=0.5),
                scene_camera=dict(up=dict(x=0, y=0, z=1), 
                                    center=dict(x=0, y=0, z=-0.1),
                                    eye=dict(x=1.5, y=-1.4, z=0.5)),
                                    margin=dict(l=0, r=0, b=0, t=0),
                scene = dict(xaxis=dict(backgroundcolor='white',
                                        color='black',
                                        gridcolor='#f0f0f0',
                                        title_font=dict(size=10),
                                        tickfont=dict(size=10),
                                        ),
                            yaxis=dict(backgroundcolor='white',
                                        color='black',
                                        gridcolor='#f0f0f0',
                                        title_font=dict(size=10),
                                        tickfont=dict(size=10),
                                        ),
                            zaxis=dict(backgroundcolor='lightgrey',
                                        color='black', 
                                        gridcolor='#f0f0f0',
                                        title_font=dict(size=10),
                                        tickfont=dict(size=10),
                                        )))

fig.update_traces(marker=dict(size=3, line=dict(color='black', width=0.1)))

fig.write_html("figs/umap_years.html")

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=df.fips, z=df.unemp,
                                    colorscale="Viridis", zmin=0, zmax=12,
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()